In [ ]:
#@title Convert all Metro PDF's to excel
# create folder metro/pdfs from your application root and upload pdfs to it

try:
  import os
  import fitz
  import tabula
  import pandas as pd
  from pathlib import Path
except Exception as e:
  !pip install PyMuPDF==1.23.7
  !pip install -q tabula-py==2.9.0
finally:
  import os
  import fitz
  import tabula
  import pandas as pd
  from pathlib import Path


Path("/content/metro/pdfs/").mkdir(parents=True, exist_ok=True)
output_filename = "metro_combined_orders.csv"
pdf_source_path = "/content/metro/pdfs/"
input("upload pdfs hit enter")
# Output Columns List
columns = [
    "Supplier no",
    "Order no",
    "Delivery date from",
    "Delivery date to",
    "Order for Store Number",
    "Article Number",
    "Subsystem number",
    "Your art-no.",
    "Sales-EAN",
    "Article Name",
    "HSN",
    "Variant",
    "MRP INR",
    "Buy Bdl",
    "Sales BDL",
    "SGST %",
    "CGST %",
    "IGST %",
    "CESS %",
    "Qty Buy Bdl",
    "Qty sale Bdl",
    "Cess INR",
    "Total NBP INR",
    "NBP pP amt",
    "Total Order Amount",
    "Total order amount INR",
]
#common indexmap
indexmap = {
    "Article Number": [1, 2],
    "Subsystem number": 3,
    "Your art-no.": 4,
    "Sales-EAN": 14,
    "Article Name": [5, 15],
    "HSN": 20,
    "Variant": 6,
    "MRP INR": 21,
    "Buy Bdl": [10, 12],
    "Sales BDL": [11, 13],
    "SGST %": 0,
    "CGST %": 0,
    "IGST %": 23,
    "CESS %": 0,
    "Qty Buy Bdl": 8,
    "Qty sale Bdl": 7,
    "Cess INR": 0,
    "Total NBP INR": 9,
    "NBP pP amt": 17,
}

def divide_chunks(l, n):
    # Dynamic list split
    for i in range(0, len(l), n):
        yield l.iloc[i : i + n]

def page_formettor(pageno, df):
  # formet table according to output format
    import re

    df_trans = []

    trans = list(divide_chunks(df, 3))
    if pageno == 0:
        for x in trans:
            indexmap = {}
            indexmap["Article Number"] = x.iloc[0].iloc[1]
            indexmap["Subsystem number"] = x.iloc[1].iloc[1]
            if (
                re.search(
                    r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{1}[A-Z]{1}\d{2}",
                    x.iloc[0].iloc[3],
                )
                or re.search(
                    r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{2}[A-Z]{1}\d{1}",
                    x.iloc[0].iloc[3],
                )
                or re.search(
                    r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{1}[A-Z]{1}", x.iloc[0].iloc[3]
                )
            ):
                indexmap["Your art-no."] = x.iloc[0].iloc[1].split(" ")[0]
            else:
                indexmap["Your art-no."] = x.iloc[0].iloc[3]
            indexmap["Sales-EAN"] = x.iloc[1].iloc[3]
            if type(x.iloc[1].iloc[4]) != float:
                indexmap["Article Name"] = x.iloc[0].iloc[4] + x.iloc[1].iloc[4]
            else:
                indexmap["Article Name"] = x.iloc[0].iloc[4]
            indexmap["HSN"] = x.iloc[2].iloc[4]
            indexmap["Variant"] = x.iloc[0].iloc[6].split(" ")[0]
            if re.search(r"^\d{2}\.\d{2}%", x.iloc[2].iloc[6].split(" ")[0]):
                indexmap["MRP INR"] = ""
            else:
                indexmap["MRP INR"] = x.iloc[2].iloc[6].split(" ")[0]
            indexmap["Buy Bdl"] = (
                x.iloc[0].iloc[6].split(" ")[1] + x.iloc[0].iloc[6].split(" ")[2]
            )
            indexmap["Sales BDL"] = (
                x.iloc[1].iloc[6].split(" ")[0] + x.iloc[1].iloc[6].split(" ")[1]
            )
            indexmap["SGST %"] = ""
            indexmap["CGST %"] = ""
            indexmap["IGST %"] = x.iloc[2].iloc[6].split(" ")[2]
            indexmap["CESS %"] = ""
            indexmap["Qty Buy Bdl"] = x.iloc[0].iloc[6].split(" ")[4]
            indexmap["Qty sale Bdl"] = x.iloc[1].iloc[6].split(" ")[3]
            indexmap["Cess INR"] = ""
            indexmap["Total NBP INR"] = x.iloc[0].iloc[6].split(" ")[-1]
            indexmap["NBP pP amt"] = x.iloc[1].iloc[6].split(" ")[-1]
            df_trans.append(indexmap)
    else:
        for x in trans:
            if len(x) == 3:
                indexmap = {}
                indexmap["Article Number"] = x.iloc[0].iloc[1]
                indexmap["Subsystem number"] = x.iloc[1].iloc[1]
                if (
                    re.search(
                        r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{1}[A-Z]{1}\d{2}",
                        x.iloc[0].iloc[3],
                    )
                    or re.search(
                        r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{2}[A-Z]{1}\d{1}",
                        x.iloc[0].iloc[3],
                    )
                    or re.search(
                        r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{1}[A-Z]{1}",
                        x.iloc[0].iloc[3],
                    )
                ):
                    indexmap["Your art-no."] = x.iloc[0].iloc[1].split(" ")[0]
                else:
                    indexmap["Your art-no."] = x.iloc[0].iloc[3]
                indexmap["Sales-EAN"] = x.iloc[1].iloc[3]
                if type(x.iloc[1].iloc[4]) != float:
                    indexmap["Article Name"] = x.iloc[0].iloc[4] + x.iloc[1].iloc[4]
                else:
                    indexmap["Article Name"] = x.iloc[0].iloc[4]
                indexmap["HSN"] = x.iloc[2].iloc[4]
                indexmap["Variant"] = x.iloc[0].iloc[5]
                if re.search(r"^\d{2}\.\d{2}%", str(x.iloc[2].iloc[5])):
                    indexmap["MRP INR"] = ""
                else:
                    indexmap["MRP INR"] = x.iloc[2].iloc[5]
                indexmap["Buy Bdl"] = x.iloc[0].iloc[6]
                indexmap["Sales BDL"] = x.iloc[1].iloc[6]
                indexmap["SGST %"] = ""
                indexmap["CGST %"] = ""
                if type(x.iloc[2].iloc[8]) != float:
                    indexmap["IGST %"] = x.iloc[2].iloc[8]
                else:
                    indexmap["IGST %"] = x.iloc[2].iloc[7]
                indexmap["CESS %"] = ""
                indexmap["Qty Buy Bdl"] = x.iloc[0].iloc[10].split(" ")[0]
                indexmap["Qty sale Bdl"] = x.iloc[1].iloc[10].split(" ")[0]
                indexmap["Cess INR"] = ""
                indexmap["Total NBP INR"] = x.iloc[0].iloc[10].split(" ")[-1]
                indexmap["NBP pP amt"] = x.iloc[1].iloc[10].split(" ")[-1]
                df_trans.append(indexmap)
    return pd.DataFrame.from_records(df_trans)

def get_trans_df(dfs):
  # get Trans dataframes
    trans_df = []
    for i, x in enumerate(dfs):
        if i == 0:
            get_all_indexs = page_formettor(i, x.iloc[5:])
        else:
            get_all_indexs = page_formettor(i, x.iloc[2:])
        trans_df.append(get_all_indexs)
    return trans_df

# getting Common Row Data
def get_row_data(pdffolderpath, pdfreader, path=False):
    missing = None
    pdfpath = []
    if missing is None:
        pdfs = os.listdir(pdffolderpath)
        files = []
        for i in pdfs:
            fit = pdfreader.open("".join([pdffolderpath, i]))
            if path:
                pdfpath.append("".join([pdffolderpath, i]))
            pages = []
            for page in fit:
                pages.append(page.get_text())
            files.append({"".join([pdffolderpath, i]): pages[:-1]})
    else:
        files = []
        pdfs = []
        for id in missing:
            for sed in os.listdir(pdffolderpath):
                if id in sed:
                    pdfs.append(sed)
        for i in pdfs:
            fit = pdfreader.open("".join([pdffolderpath, i]))
            pages = []
            for page in fit:
                pages.append(page.get_text())
            files.append({"".join([pdffolderpath, i]): pages[:-1]})
    if path:
        return files, pdfpath
    else:
        return files

def get_pdf_folder_done(folder_path):
  # get all Pdfs and tables
    total_df_data = []
    for i in os.listdir(folder_path):
        dfs = tabula.read_pdf(os.path.join(folder_path, i), pages="all")
        df = pd.concat(get_trans_df(dfs))
        df["file"] = os.path.join(folder_path, i)
        total_df_data.append(df)
    return pd.concat(total_df_data)

files = get_row_data(pdf_source_path, fitz)
df    = get_pdf_folder_done(pdf_source_path)

invoices = []
for i, v in enumerate(files):
    file, v = list(v.keys())[0], list(v.values())[0]
    fipag = []
    invdetails = {}
    for q, c in enumerate(v):
        if q == 0:
            if [x for x in v[q].split("\n") if x.startswith("Supplier no")] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("Supplier no")][0]
                )
                invdetails["Supplier no"] = testa[indexs + 1]
            if [x for x in v[q].split("\n") if x.startswith("Order no")] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("Order no")][0]
                )
                invdetails["Order no"] = testa[indexs + 1]
            if [
                x for x in v[q].split("\n") if x.startswith("Delivery date from")
            ] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("Delivery date from")][
                        0
                    ]
                )
                invdetails["Delivery date from"] = testa[indexs - 1]
            if [x for x in v[q].split("\n") if x.startswith("to")] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("to")][0]
                )
                invdetails["Delivery date to"] = testa[indexs - 1]
            if [
                x for x in v[q].split("\n") if x.startswith("Order for Store no")
            ] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("Order for Store no")][
                        0
                    ]
                )
                invdetails["Order for Store Number"] = (
                    testa[indexs].split(":")[-1].strip()
                )
            valid = [q for x in c.split("\n") if "Total order amount" in x]
            if valid == []:
                fipag.append(c.split("\n")[57:])
            else:
                raise NotImplementedError("data sample not available")
        else:
            data = None
            valid = [q for x in c.split("\n") if "Total order amount" in x]
            if valid == []:
                fipag.append(c.split("\n")[52:])
            else:
                try:
                    data = float(c.split("\n")[67:][0])
                except ValueError as e:
                    fipag.append(c.split("\n")[68:])
                if data is not None:
                    fipag.append(c.split("\n")[67:])
                invdetails["Total order amount"] = c.split("\n")[52:][0]
    invdetails["file"] = file
    invoices.append(invdetails)

invoicedf = pd.DataFrame.from_records(invoices)
records = []
# prepearing output
for k, v in df.iterrows():
    common_info = invoicedf[invoicedf["file"] == v["file"]]
    for x, d in common_info.iterrows():
        for xs in common_info.columns.to_list():
            v[xs] = d[xs]
    records.append(v.to_dict())

df = pd.DataFrame.from_records(records).merge(invoicedf, on='file', how='left')
df=df.fillna(0)
try:
  dfd=df[df['Total order amount']==0]
except KeyError as E:
  df['Total order amount']=[0 for x in range(len(df))]
  dfd=df[df['Total order amount']==0]
filllastdata=[]
for k,v in dfd.iterrows():
  pdf=fitz.open(v['file'])
  content=[]
  for page in pdf:
    content.append(page.get_text())
  totalorder=content[-1].split("\n")
  filllastdata.append((v['file'],totalorder[totalorder.index('Total order amount INR')-1]))

for x in filllastdata:
  df.loc[df['file']==x[0], 'Total order amount'] = x[1]

df = df.drop(columns=['file'])
df.to_csv(output_filename, index=False)



In [ ]:
#@title Metro relience pdfs

try:
   import pandas as pd
   import pdfplumber
   import os
   from pathlib import Path
except ImportError as E:
  %pip install pdfplumber==0.10.3
finally:
  import pandas as pd
  import pdfplumber
  import os
  from pathlib import Path

basepdfforderpath="/content/metroREL/"

Path(basepdfforderpath).mkdir(parents=True, exist_ok=True)


outputfilename="testfile.xlsx"
input("upload pdfs to {} and hit enter".format(basepdfforderpath))
def get_all_pdfs_data(basefolderpath):
    file_groups     =  None
    filesdf         =  []
    owe             = [x for x in os.listdir(basefolderpath) if x.endswith(".pdf") or x.endswith(".PDF")]
    for x in owe:
        pdf         = pdfplumber.open(os.path.join(basefolderpath,x))
        table       = pdf.pages
        file_groups = table
        rowdata     = file_groups[0].extract_tables()[0]
        file_groups=file_groups[1:-10]
        if len(file_groups)>0:
           pass
        else:
           file_groups=table[1:-9]
        if "Total Order Value:" not in file_groups[-1].extract_tables()[0][-1]:
          if 'TOTAL BASIC VALUE' in file_groups[-1].extract_tables()[0][-1]:
            dummylist=file_groups[-1].extract_tables()[0][-1]
            pendint=table[1:-9][-1].extract_text().split("\n")[3:5]
            appendabkle=[]
            for xs in pendint:
              pendi=[" ".join(xs.split(" ")[0:-2]),xs.split(" ")[-1]]
              du=dummylist
              du[4]=pendi[0]
              du[-1]=pendi[-1]
              appendabkle.append(du)
            sq=file_groups[-1].extract_tables()
            for xsw in appendabkle:
              sq[0].append(xsw)
            file_groups[-1]=sq
          else:
            file_groups=table[1:-9]
        formatted_values=[]
        for xi in file_groups:
           if type(xi)==list:
            page=xi
           else:
            page=xi.extract_tables()
           formatted_values.append(page)
        value_added=[]
        for iss,xss in enumerate(formatted_values):
           base=xss[0][0]
           base.append("PO Number")
           xss[0][0]=base
           ba_values=[]
           for xqw in range(len(xss[0])):
              if xqw>0:
                 basw=xss[0][xqw]
                 basw.append(x.split("_")[1].split(".")[0])
                 xss[0][xqw]=basw
                 teste={}
                 for sw in range(len(base)):
                    teste[base[sw]]=basw[sw]
                 ba_values.append(teste)
           value_added.append(pd.DataFrame.from_records(ba_values))
        filesdf.append(pd.concat(value_added))
    return filesdf


file_groups=get_all_pdfs_data(basepdfforderpath)
def format_df(list_df):
    list_ = []
    for i, x in enumerate(list_df):
        listrows = [len(x) - 4, len(x) - 3, len(x) - 2, len(x) - 1]
        row_cols = x.iloc[listrows]
        if row_cols.iloc[0].loc["Sr.No"] != "Grand Total of Qty":
            listrows = [len(x) - 5, len(x) - 4, len(x) - 3, len(x) - 2, len(x) - 1]
            row_cols = x.iloc[listrows]
        if row_cols.iloc[0].loc["Sr.No"] != "Grand Total of Qty":
           pass
        maindf = x.iloc[0 : len(x) - len(listrows)]
        update_cols = {
            "Grand Total of Qty": [
                row_cols.iloc[0].loc["Quantity"] for u in range(len(maindf))
            ],
            "TOTAL BASIC VALUE": [
                row_cols.iloc[1].loc["Total Base Value"] for u in range(len(maindf))
            ],
            "TOTAL IGST": [
                row_cols.iloc[2].loc["Total Base Value"] for u in range(len(maindf))
            ],
            " Total Order Value": [
                row_cols.iloc[3].loc["Total Base Value"] for u in range(len(maindf))
            ],
        }
        for k, v in update_cols.items():
            maindf.insert(len(maindf.columns.to_list()) - 2, k, v)
        list_.append(maindf)
    return list_


def final_df_formatter(df):
    dfs = None
    value_dict = {}
    try:
        dfs = df[
            [
                "IGST (%)\nCESS(%)",
                "IGST\nCESS",
                "CGST (%)\nSGST(%)\nCESS(%)",
                "CGST\nSGST\nCESS",
            ]
        ]
    except Exception as E:
        dfs = df[["IGST (%)\nCESS(%)", "IGST\nCESS"]]
    if dfs is not None:
        if len(dfs.columns.to_list()) >= 2:
            df = df.fillna("01")
            value_dict = {
                "IGST (%)": [],
                "CESS(%)": [],
                "CGST (%)": [],
                "SGST(%)": [],
                "IGST": [],
                "CGST": [],
                "SGST": [],
                "CESS": [],
            }
            for i, v in df.iterrows():
                if v["IGST (%)\nCESS(%)"] != "01":
                    if type(v["IGST (%)\nCESS(%)"]) == str:
                        i, c = v["IGST (%)\nCESS(%)"].split("\n")
                    else:
                        i, c = "0", "0"
                    value_dict["IGST (%)"].append(i)
                    value_dict["CESS(%)"].append(c)
                    io, co = v["IGST\nCESS"].split("\n")
                    value_dict["IGST"].append(io)
                    value_dict["CESS"].append(co)
                    value_dict["CGST (%)"].append("0")
                    value_dict["SGST(%)"].append("0")
                    value_dict["CGST"].append("0")
                    value_dict["SGST"].append("0")
                elif v["CGST (%)\nSGST(%)\nCESS(%)"] != "01":
                    c, s, cc = v["CGST (%)\nSGST(%)\nCESS(%)"].split("\n")
                    value_dict["IGST (%)"].append("0")
                    value_dict["CESS(%)"].append(cc)
                    co, so, cco = v["CGST\nSGST\nCESS"].split("\n")
                    value_dict["IGST"].append("0")
                    value_dict["CESS"].append(cco)
                    value_dict["CGST (%)"].append(c)
                    value_dict["SGST(%)"].append(s)
                    value_dict["CGST"].append(co)
                    value_dict["SGST"].append(so)
    columns_to_select = list(set(df.columns.to_list()) - set(dfs.columns.to_list()))
    finaldf = df[columns_to_select]
    for k, v in value_dict.items():
        finaldf.insert(len(finaldf.columns.to_list()) - 2, k, v)
    return finaldf


def divide_combined_values(df):
    cols = df.columns.to_list()
    df_list = []
    for k, v in df.iterrows():
        dellis = []
        for aq in cols:
            if "\n" in v[aq] and "\n" in aq:
                col = aq.split("\n")
                val = v[aq].split("\n")
                if len(col) == len(val):
                    for i, xs in enumerate(col):
                        v[xs] = val[i]
                dellis.append(aq)
            elif "\n" in v[aq] and "\n" not in aq:
                col = [aq + "_" + str(x) for x in range(len(v[aq].split("\n")))]
                val = v[aq].split("\n")
                if len(col) == len(val):
                    for i, xs in enumerate(col):
                        v[xs] = val[i]
                dellis.append(aq)

        v = v.drop(labels=dellis)
        df_list.append(v.to_dict())
    return pd.DataFrame.from_records(df_list)

def patch_exceptions(df, colstoremove, value_columns):
    df = df.fillna("02")
    query = ""
    for i, x in enumerate(colstoremove):
        if i == 0:
            query += x
            query += "!="
            query += '"02"'
        else:
            query += " & "
            query += x
            query += "!="
            query += '"02"'
    try:
      index = df.query(query).index.to_list()
    except Exception as e:
      index=None

    if index is not None:
      for xs, d in enumerate(index):
        for io, xsd in enumerate(value_columns):
          df.iloc[d].loc[xsd] = df.iloc[d].loc[colstoremove[io]]
    df = df.replace(["02"], "0")
    try:
      df = df.drop(colstoremove, axis=1)
    except Exception as e:
      df=df
    return df

def validate_financials(df):
  valida                             =   {}
  for x in range(len(df)):
    subtotal                         =   float(df.iloc[x].loc['Quantity_0'])*float(df.iloc[x].loc["Base Cost"] if "," not in df.iloc[x].loc["Base Cost"] else df.iloc[x].loc["Base Cost"].replace(",",""))

    if float(round(subtotal,2))     ==   float(df.iloc[x].loc["Total Base Value"] if "," not in df.iloc[x].loc["Total Base Value"] else df.iloc[x].loc["Total Base Value"].replace(",","")):
      pass
    else:
      valida["subtotal"]            =   (x,subtotal,float(df.iloc[x].loc["Total Base Value"] if "," not in df.iloc[x].loc["Total Base Value"] else df.iloc[x].loc["Total Base Value"].replace(",","")))
  groups=df.groupby("PO Number")
  for name,group in groups:
    values                          =   [float(x if "," not in x else x.replace(",","")) for x in group["Total Base Value"].values.tolist()]
    if float(round(sum(values),2)) ==  float(group.iloc[0].loc["TOTAL BASIC VALUE"] if "," not in group.iloc[0].loc["TOTAL BASIC VALUE"] else group.iloc[0].loc["TOTAL BASIC VALUE"].replace(",","")):
      pass
    else:
      valida["TOTAL BASIC VALUE"]         =  (name,float(round(sum(values),2)),float(group.iloc[0].loc["TOTAL BASIC VALUE"] if "," not in group.iloc[0].loc["TOTAL BASIC VALUE"] else group.iloc[0].loc["TOTAL BASIC VALUE"].replace(",","")))
  return valida,df

valid,df=validate_financials(patch_exceptions(
    divide_combined_values(final_df_formatter(pd.concat(format_df(get_all_pdfs_data(basepdfforderpath))))),
    ["MRP", "UOM", "Quantity"],
    ["MRP_0", "UOM_0", "Quantity_0"],
))
def format_exception(df):
  cols=[x for x in df.columns.to_list() if "\n" in x]
  if cols!=[]:
    df.rename(columns = {cols[0]:cols[0].split("\n")[0]},inplace=True)
    if len(cols[0].split("\n"))>1:
      for z in cols[0].split("\n")[1:]:
        df[z]=[" " for d in range(len(df))]
  return df
if valid=={}:
  print("validated subtotal,Grand total with 0 errors")
  df=format_exception(df)
  cols=df.columns.to_list()
  orders=",".join(
    [
        str(cols.index("Sr.No")),
        str(cols.index("Article No.")),
        str(cols.index("HSN Code")),
        str(cols.index("EAN No.")),
        str(cols.index("Vendor Article No.")),
        str(cols.index("Vendor Item No")),
        str(cols.index("Material Description")),
        str(cols.index("Delivery Date")),
        str(cols.index("Site")),
        str(cols.index("Quantity_0")),
        str(cols.index("Quantity_1")),
        str(cols.index("UOM_0")),
        str(cols.index("UOM_1")),
        str(cols.index("MRP_0")),
        str(cols.index("MRP_1")),
        str(cols.index("Base Cost")),
        str(cols.index("IGST (%)")),
        str(cols.index("CESS(%)")),
        str(cols.index("CGST (%)")),
        str(cols.index("SGST(%)")),
        str(cols.index("IGST")),
        str(cols.index("CGST")),
        str(cols.index("SGST")),
        str(cols.index("CESS")),
        str(cols.index("Total Base Value")),
        str(cols.index("TOTAL IGST")),
        str(cols.index("Grand Total of Qty")),
        str(cols.index("TOTAL BASIC VALUE")),
        str(cols.index("Total Order Value")),
        str(cols.index("PO Number")),


    ]
)
  df=df.loc[:,[df.columns.to_list()[int(s)]for s in orders.split(",")]]
  df.to_excel(outputfilename,index=False)
  print("file generated")
else:
  validated_with_diff=[]
  for k,v in valid.items():
    diff=v[1]-v[2]
    if diff <=1:
      validated_with_diff.append(True)
    else:
      validated_with_diff.append(False)

  if valid!={} and False not in validated_with_diff:
      df=format_exception(df)
      cols=df.columns.to_list()
      orders=",".join(
    [
        str(cols.index("Sr.No")),
        str(cols.index("Article No.")),
        str(cols.index("HSN Code")),
        str(cols.index("EAN No.")),
        str(cols.index("Vendor Article No.")),
        str(cols.index("Vendor Item No")),
        str(cols.index("Material Description")),
        str(cols.index("Delivery Date")),
        str(cols.index("Site")),
        str(cols.index("Quantity_0")),
        str(cols.index("Quantity_1")),
        str(cols.index("UOM_0")),
        str(cols.index("UOM_1")),
        str(cols.index("MRP_0")),
        str(cols.index("MRP_1")),
        str(cols.index("Base Cost")),
        str(cols.index("IGST (%)")),
        str(cols.index("CESS(%)")),
        str(cols.index("CGST (%)")),
        str(cols.index("SGST(%)")),
        str(cols.index("IGST")),
        str(cols.index("CGST")),
        str(cols.index("SGST")),
        str(cols.index("CESS")),
        str(cols.index("Total Base Value")),
        str(cols.index("TOTAL IGST")),
        str(cols.index("Grand Total of Qty")),
        str(cols.index("TOTAL BASIC VALUE")),
        str(cols.index(' Total Order Value')),
        str(cols.index("PO Number")),


    ]
)
      df=df.loc[:,[df.columns.to_list()[int(s)] for s in orders.split(",")]]
      df.to_excel(outputfilename,index=False)
      print("file generated")
      print("validated",validated_with_diff)
  else:
    print(valid,validated_with_diff)